IT5437 – Assignment 1

In [ ]:
# === Setup: installs & imports ===
!pip -q install opencv-python-headless==4.10.0.84

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

plt.rcParams['figure.figsize'] = (6,4)

def load_image(path):
    """Load an image from path (returns BGR)."""
    img = cv.imread(str(path), cv.IMREAD_COLOR)
    if img is None:
        raise FileNotFoundError(f"Image not found: {path}")
    return img

def show(img, title=None, cmap=None):
    plt.figure()
    if img.ndim == 2:
        plt.imshow(img, cmap=cmap or 'gray')
    else:
        plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    if title:
        plt.title(title)
    plt.axis("off")
    plt.show()

def plot_hist(img, title="Histogram", color=True):
    plt.figure()
    if img.ndim == 2:
        plt.hist(img.ravel(), bins=256, range=(0,255))
    else:
        if color:
            for i, col in enumerate(("b","g","r")):
                plt.hist(img[:,:,i].ravel(), bins=256, range=(0,255), alpha=0.5, label=col)
            plt.legend()
        else:
            gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            plt.hist(gray.ravel(), bins=256, range=(0,255))
    plt.title(title)
    plt.show()

def to_uint8(x):
    return np.clip(np.round(x), 0, 255).astype(np.uint8)

Q1. Intensity transformation (apply Fig. 1a to Fig. 1b) Define the curve by control points and apply via LUT.

In [ ]:
# Q1 – Intensity Transformation
img = load_image(r"emma.jpg")
show(img, "Q1: Input")

In [ ]:
# Define control points (adjust to match Fig. 1a)
control_x = np.array([0, 50, 100, 150, 200, 255], dtype=np.float32)
control_y = np.array([0, 30, 120, 180, 220, 255], dtype=np.float32)

x = np.arange(256, dtype=np.float32)
lut = np.interp(x, control_x, control_y)
lut_u8 = to_uint8(lut)

out = cv.LUT(img, lut_u8)
show(out, "Q1: Transformed")

plt.figure()
plt.plot(x, lut, lw=2)
plt.title("Q1: Intensity Transformation Curve")
plt.xlabel("Input intensity")
plt.ylabel("Output intensity")
plt.grid(True)
plt.show()

Q2. Accentuate white & gray matter (Fig. 2) Use separate LUTs to emphasize high (white matter) and mid (gray matter) intensities.

In [ ]:
# Q2 – Brain proton density image (Fig. 2)
brain = load_image("brain_proton_density_slice.png")  
# Convert to grayscale if image is RGB
if brain.ndim == 3:
    brain_gray = cv.cvtColor(brain, cv.COLOR_BGR2GRAY)
else:
    brain_gray = brain.copy()

# Show result
show(brain_gray, "Q2: Brain (gray)")

In [ ]:
wm_low, wm_high = 140, 255
x = np.arange(256, dtype=np.float32)
wm_lut = np.piecewise(x,
    [x<wm_low, (x>=wm_low)&(x<=wm_high), x>wm_high],
    [0, lambda t: 255*(t-wm_low)/(wm_high-wm_low+1e-6), 255])
wm = cv.LUT(brain_gray, to_uint8(wm_lut))

gm_c, gm_w = 120, 60
gm_low, gm_high = gm_c-gm_w, gm_c+gm_w
gm_lut = np.piecewise(x,
    [x<gm_low, (x>=gm_low)&(x<=gm_high), x>gm_high],
    [0, lambda t: 255*(t-gm_low)/(gm_high-gm_low+1e-6), 0])
gm = cv.LUT(brain_gray, to_uint8(gm_lut))

show(wm, "Q2: White matter accentuated")
show(gm, "Q2: Gray matter accentuated")
plt.figure(); plt.plot(x, wm_lut); plt.title("Q2: WM LUT"); plt.grid(True); plt.show()
plt.figure(); plt.plot(x, gm_lut); plt.title("Q2: GM LUT"); plt.grid(True); plt.show()